## 결측치 제거(출,도착 좌표 존재하지 않는 경우)

In [ ]:
import pandas as pd

# CSV 파일 읽기
input_file = "세종대_출도착_240530_국적추가.csv"
output_file = "input.csv"

# CSV 파일 읽기 (UTF-8로 읽기)
df = pd.read_csv(input_file, encoding='utf-8')

# 결측값이 있는 행 삭제
df_cleaned = df.dropna(subset=['출발X좌표', '출발Y좌표', '도착X좌표', '도착Y좌표'])

# 새로운 파일로 저장 (UTF-8로 저장)
df_cleaned.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"결측값이 있는 행이 제거되었으며, 결과는 {output_file}에 저장되었습니다.")


결측값이 있는 행이 제거되었으며, 결과는 input.csv에 저장되었습니다.


In [31]:
import pandas as pd
import requests
import csv

# OSRM 외부 API 주소
osrm_url = "http://router.project-osrm.org/route/v1/foot/"

# CSV 파일 경로
input_file = "input.csv"  # 입력 파일 경로
output_file = "output.csv"  # 출력 파일 경로

# 경로 데이터를 저장할 리스트
output_data = []

# 입력 CSV 파일 읽기
df = pd.read_csv(input_file, encoding='utf-8')

# 출력 파일에 필요한 열 정의 ('내국인', '단기외국인', '장기외국인', 'MALE', 'FEMALE', '1' ~ '14' 추가)
output_columns = ['일자', '출발시간', 'X좌표', 'Y좌표', '총 유동인구', '내국인', '단기외국인', '장기외국인', 'MALE', 'FEMALE'] + [str(i) for i in range(1, 15)]

# 결과 저장용 DataFrame 생성
output_df = pd.DataFrame(columns=output_columns)

# 입력 데이터 처리
for idx, row in df.iterrows():

    # 좌표 설정
    start_coords = f"{row['출발X좌표']},{row['출발Y좌표']}"
    end_coords = f"{row['도착X좌표']},{row['도착Y좌표']}"

    # OSRM API 호출
    response = requests.get(f"{osrm_url}{start_coords};{end_coords}?geometries=geojson")

    if response.status_code == 200:
        data = response.json()

        # 경로의 좌표들 추출
        coordinates = data['routes'][0]['geometry']['coordinates']

        # 각 좌표에 대해 '일자', '출발시간', 'X좌표', 'Y좌표', '총 유동인구', '내국인', '단기외국인', '장기외국인', 'MALE', 'FEMALE', '1' ~ '14' 처리
        for coord in coordinates:
            x, y = coord  # X좌표, Y좌표

            # 해당 좌표와 일자, 출발시간이 이미 존재하는지 확인
            existing_row = output_df[(output_df['X좌표'] == x) &
                                     (output_df['Y좌표'] == y) &
                                     (output_df['일자'] == row['일자']) &
                                     (output_df['출발시간'] == row['출발시간'])]

            if not existing_row.empty:
                # 이미 있으면 '총 유동인구' 값 더하기
                output_df.loc[existing_row.index, '총 유동인구'] += row['유동인구']

                # 국적에 따라 '내국인', '단기외국인', '장기외국인' 값 더하기
                if row['국적'] == '내국인':
                    output_df.loc[existing_row.index, '내국인'] += row['유동인구']
                elif row['국적'] == '단기외국인':
                    output_df.loc[existing_row.index, '단기외국인'] += row['유동인구']
                elif row['국적'] == '장기외국인':
                    output_df.loc[existing_row.index, '장기외국인'] += row['유동인구']

                # 성별에 따라 'MALE', 'FEMALE' 값 더하기
                if row['성'] == 'MALE':
                    output_df.loc[existing_row.index, 'MALE'] += row['유동인구']
                elif row['성'] == 'FEMALE':
                    output_df.loc[existing_row.index, 'FEMALE'] += row['유동인구']

                # 연령에 따라 '1' ~ '14' 값 더하기
                age_column = str(row['연령'])
                if age_column in output_df.columns:
                    output_df.loc[existing_row.index, age_column] += row['유동인구']
            else:
                # 새로운 좌표라면 '내국인', '단기외국인', '장기외국인', 'MALE', 'FEMALE', '1' ~ '14' 값 추가
                new_row = {
                    '일자': row['일자'],
                    '출발시간': row['출발시간'],
                    'X좌표': x,
                    'Y좌표': y,
                    '총 유동인구': row['유동인구'],
                    '내국인': row['유동인구'] if row['국적'] == '내국인' else 0,
                    '단기외국인': row['유동인구'] if row['국적'] == '단기외국인' else 0,
                    '장기외국인': row['유동인구'] if row['국적'] == '장기외국인' else 0,
                    'MALE': row['유동인구'] if row['성'] == 'MALE' else 0,
                    'FEMALE': row['유동인구'] if row['성'] == 'FEMALE' else 0,
                }

                # 연령에 맞는 열을 초기화
                for i in range(1, 15):
                    new_row[str(i)] = row['유동인구'] if row['연령'] == i else 0

                new_row_df = pd.DataFrame(new_row, index=[0])
                output_df = pd.concat([output_df, new_row_df], ignore_index=True)

# 중복처리 후 결과를 CSV 파일로 저장
output_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"경로 데이터가 {output_file}에 저장되었습니다.")


<ipython-input-31-568728a1334c>:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, new_row_df], ignore_index=True)


경로 데이터가 output.csv에 저장되었습니다.
